In [ ]:
import pygame
import Box2D
from Box2D.b2 import world, polygonShape
import nbimporter
from humanoid import Humanoid

class Simulation:
    def __init__(self):
        pygame.init()
        self.width, self.height = 1600, 600
        self.screen = pygame.display.set_mode((self.width, self.height))
        pygame.display.set_caption("2D Humanoid Simulation")

        # Pixels per meter (scaling factor)
        self.ppm = 100

        # Colors
        self.bg_color = (255, 255, 255)
        self.ground_color = (0, 0, 0)

        # Load flag image and scale it
        self.flag_image = pygame.image.load('assets/flag.png')
        self.flag_image = pygame.transform.scale(self.flag_image, (50, 50))  # Resize to make it small

        # Create the Box2D world
        self.world = world(gravity=(0, -10), doSleep=True)

        # Create ground
        self.ground_height = 50 / self.ppm
        self.ground = self.world.CreateStaticBody(
            position=(self.width / (2 * self.ppm), self.ground_height / 2),
            shapes=polygonShape(box=(self.width / (2 * self.ppm), self.ground_height / 2))
        )

        # Create humanoid
        self.humanoid = Humanoid(self.world, x=(self.width-1200) / (2 * self.ppm), y=self.height / self.ppm - self.ground_height)

    def render_ground(self):
        """Render the ground."""
        pygame.draw.rect(
            self.screen,
            self.ground_color,
            pygame.Rect(0, self.height - int(self.ground_height * self.ppm), self.width, int(self.ground_height * self.ppm)),
        )

    def render_flag(self):
        """Render the flag image at the right of the window at ground level."""
        flag_x = self.width - self.flag_image.get_width() - 10  # 10px from the right edge
        flag_y = self.height - int(self.ground_height * self.ppm) - self.flag_image.get_height()
        self.screen.blit(self.flag_image, (flag_x, flag_y))

    def run(self):
        clock = pygame.time.Clock()
        running = True

        while running:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    running = False

            # Clear the screen
            self.screen.fill(self.bg_color)

            # Render ground and flag
            self.render_ground()
            self.render_flag()

            # Render humanoid
            self.humanoid.render(self.screen, self.ppm)

            # Set motor speeds (adjusted for stability during debugging)
            self.humanoid.update_motors([1, -1, 0.5, -0.5])

            # Print debug info and terminate if humanoid crosses finish
            print(self.humanoid.log_state())
            if self.humanoid.torso.position.x > 16:
                print("Finish line crossed!")
                break

            # Step the simulation
            self.world.Step(1.0 / 60.0, 6, 2)

            # Update display
            pygame.display.flip()
            clock.tick(60)

        pygame.quit()


if __name__ == "__main__":
    sim = Simulation()
    sim.run()